In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# Pipelining estimators

In this section we study how different estimators maybe be chained.

## A simple example: feature extraction and selection before an estimator

### Feature extraction: vectorizer

For some types of data, for instance text data, a feature extraction step must be applied to convert it to numerical features.
To illustrate we load the SMS spam dataset we used earlier.

In [2]:
import os

with open(os.path.join("datasets", "smsspam", "SMSSpamCollection")) as f:
    lines = [line.strip().split("\t") for line in f.readlines()]
text = [x[1] for x in lines]
y = [x[0] == "ham" for x in lines]

In [3]:
from sklearn.model_selection import train_test_split

text_train, text_test, y_train, y_test = train_test_split(text, y)

Previously, we applied the feature extraction manually, like so:

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

vectorizer = TfidfVectorizer()
vectorizer.fit(text_train)

X_train = vectorizer.transform(text_train)
X_test = vectorizer.transform(text_test)

clf = LogisticRegression()
clf.fit(X_train, y_train)

clf.score(X_test, y_test)

0.96987087517934

The situation where we learn a transformation and then apply it to the test data is very common in machine learning.
Therefore scikit-learn has a shortcut for this, called pipelines:

In [5]:
from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(TfidfVectorizer(), LogisticRegression())
pipeline.fit(text_train, y_train)
pipeline.score(text_test, y_test)

0.96987087517934

As you can see, this makes the code much shorter and easier to handle. Behind the scenes, exactly the same as above is happening. When calling fit on the pipeline, it will call fit on each step in turn.

After the first step is fit, it will use the ``transform`` method of the first step to create a new representation.
This will then be fed to the ``fit`` of the next step, and so on.
Finally, on the last step, only ``fit`` is called.

![pipeline](figures/pipeline.svg)

If we call ``score``, only ``transform`` will be called on each step - this could be the test set after all! Then, on the last step, ``score`` is called with the new representation. The same goes for ``predict``.

Building pipelines not only simplifies the code, it is also important for model selection.
Say we want to grid-search C to tune our Logistic Regression above.

Let's say we do it like this:

In [6]:
# This illustrates a common mistake. Don't use this code!
from sklearn.model_selection import GridSearchCV

vectorizer = TfidfVectorizer()
vectorizer.fit(text_train)

X_train = vectorizer.transform(text_train)
X_test = vectorizer.transform(text_test)

clf = LogisticRegression()
grid = GridSearchCV(clf, param_grid={'C': [.1, 1, 10, 100]}, cv=5)
grid.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [0.1, 1, 10, 100]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=0)

### 2.1.2 What did we do wrong?

Here, we did grid-search with cross-validation on ``X_train``. However, when applying ``TfidfVectorizer``, it saw all of the ``X_train``,
not only the training folds! So it could use knowledge of the frequency of the words in the test-folds. This is called "contamination" of the test set, and leads to too optimistic estimates of generalization performance, or badly selected parameters.
We can fix this with the pipeline, though:

In [7]:
from sklearn.model_selection import GridSearchCV

pipeline = make_pipeline(TfidfVectorizer(), 
                         LogisticRegression())

grid = GridSearchCV(pipeline,
                    param_grid={'logisticregression__C': [.1, 1, 10, 100]}, cv=5)

grid.fit(text_train, y_train)
grid.score(text_test, y_test)

0.9892395982783357

Note that we need to tell the pipeline where at which step we wanted to set the parameter ``C``.
We can do this using the special ``__`` syntax. The name before the ``__`` is simply the name of the class, the part after ``__`` is the parameter we want to set with grid-search.

<img src="figures/pipeline_cross_validation.svg" width="50%">

Another benefit of using pipelines is that we can now also search over parameters of the feature extraction with ``GridSearchCV``:

In [8]:
from sklearn.model_selection import GridSearchCV

pipeline = make_pipeline(TfidfVectorizer(), LogisticRegression())

params = {'logisticregression__C': [.1, 1, 10, 100],
          "tfidfvectorizer__ngram_range": [(1, 1), (1, 2), (2, 2)]}
grid = GridSearchCV(pipeline, param_grid=params, cv=5)
grid.fit(text_train, y_train)
print(grid.best_params_)
grid.score(text_test, y_test)

{'logisticregression__C': 100, 'tfidfvectorizer__ngram_range': (1, 2)}


0.990674318507891

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
      <li>
      Create a pipeline out of a StandardScaler and Ridge regression and apply it to the Boston housing dataset (load using ``sklearn.datasets.load_boston``). Try adding the ``sklearn.preprocessing.PolynomialFeatures`` transformer as a second preprocessing step, and grid-search the degree of the polynomials (try 1, 2 and 3).
      </li>
    </ul>
</div>

In [15]:
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import make_pipeline

data = load_boston()
X, y = data.data, data.target

In [16]:
print(X.shape, y.shape)
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    random_state=1234)

(506, 13) (506,)


In [29]:
print(X_train.shape, X_test.shape)

(379, 13) (127, 13)


In [24]:
pipeline = make_pipeline(StandardScaler(), PolynomialFeatures(), Ridge())
grid = {'polynomialfeatures__degree':[1,2,3]}
reg = GridSearchCV(pipeline, param_grid=grid, verbose=3, cv=5)
reg.fit(X_train, y_train)
print(reg.best_params_)
print("score: %f"%(reg.score(X_test, y_test)))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV] polynomialfeatures__degree=1 ....................................
[CV] ........... polynomialfeatures__degree=1, score=0.627488 -   0.0s
[CV] polynomialfeatures__degree=1 ....................................
[CV] ........... polynomialfeatures__degree=1, score=0.800297 -   0.0s
[CV] polynomialfeatures__degree=1 ....................................
[CV] ........... polynomialfeatures__degree=1, score=0.762001 -   0.0s
[CV] polynomialfeatures__degree=1 ....................................
[CV] ........... polynomialfeatures__degree=1, score=0.644651 -   0.0s
[CV] polynomialfeatures__degree=1 ....................................
[CV] ........... polynomialfeatures__degree=1, score=0.535308 -   0.0s
[CV] polynomialfeatures__degree=2 ....................................
[CV] ........... polynomialfeatures__degree=2, score=0.740780 -   0.0s
[CV] polynomialfeatures__degree=2 ....................................
[CV] ........... 

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s



[CV] polynomialfeatures__degree=3 ....................................
[CV] .......... polynomialfeatures__degree=3, score=-1.227859 -   0.0s
{'polynomialfeatures__degree': 2}
score: 0.891209


[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.2s finished


In [30]:
# %load solutions/15A_ridge_grid.py
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_boston
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge

boston = load_boston()
text_train, text_test, y_train, y_test = train_test_split(boston.data,
                                                          boston.target,
                                                          test_size=0.25,
                                                          random_state=123)

pipeline = make_pipeline(StandardScaler(),
                         PolynomialFeatures(),
                         Ridge())

grid = GridSearchCV(pipeline,
                    param_grid={'polynomialfeatures__degree': [1, 2, 3]}, cv=5)

grid.fit(text_train, y_train)

print('best parameters:', grid.best_params_)
print('best score:', grid.best_score_)
print('test score:', grid.score(text_test, y_test))


best parameters: {'polynomialfeatures__degree': 2}
best score: 0.8176389414974885
test score: 0.8313120138601877
